In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import *
from collections import *
import collections
import heapq

## 1. Recommendation sytem 
Implementing a recommendation system is critical for businesses and digital platforms that want to thrive in today's competitive environment. These systems use data-driven personalization to tailor content, products, and services to individual user preferences. The latter improves user engagement, satisfaction, retention, and revenue through increased sales and cross-selling opportunities. In this section, you will attempt to implement a recommendation system by identifying similar users' preferences and recommending movies they watch to the study user. 

To be more specific, you will implement your version of the [**LSH algorithm**](https://www.learndatasci.com/tutorials/building-recommendation-engine-locality-sensitive-hashing-lsh-python/), which will take as input the user's preferred genre of movies, find the most similar users to this user, and recommend the most watched movies by those who are more similar to the user. 

__Data__: The data you will be working with can be found [here](https://www.kaggle.com/datasets/vodclickstream/netflix-audience-behaviour-uk-movies).

Looking at the data, you can see that there is data available for each user for the movies the user <ins>clicked on</ins>. Gather the __title and genre__ of the __maximum top 10 movies__ that each user clicked on regarding the __number of clicks__.


In [2]:
df = pd.read_csv('/home/theballer/Desktop/Sapienza Courses/ADM/ADM-HW4-Dataset/vodclickstream_uk_movies_03.csv')

In [3]:
df.head()

,Unnamed: 0,datetime,duration,title,genres,release_date,movie_id,user_id
0,58773,2017-01-01 01:15:09,0.0,"Angus, Thongs and Perfect Snogging","Comedy, Drama, Romance",2008-07-25,26bd5987e8,1dea19f6fe
1,58774,2017-01-01 13:56:02,0.0,The Curse of Sleeping Beauty,"Fantasy, Horror, Mystery, Thriller",2016-06-02,f26ed2675e,544dcbc510
2,58775,2017-01-01 15:17:47,10530.0,London Has Fallen,"Action, Thriller",2016-03-04,f77e500e7a,7cbcc791bf
3,58776,2017-01-01 16:04:13,49.0,Vendetta,"Action, Drama",2015-06-12,c74aec7673,ebf43c36b6
4,58777,2017-01-01 19:16:37,0.0,The SpongeBob SquarePants Movie,"Animation, Action, Adventure, Comedy, Family, ...",2004-11-19,a80d6fc2aa,a57c992287


### Top movies clicked by the users in total.

So, here I have grouped by title and genres first and used count method, after I sorted by duration, which is basically sorting by count number, not by the values of duration itself, because I used group by on them all the columns will have the count number, instead of regular numbers.

In [4]:
df.groupby(by=['title', 'genres']).count().sort_values(by='duration', ascending=False).reset_index()[['title', 'genres']].head(10)

,title,genres
0,Black Mirror: Bandersnatch,"Drama, Mystery, Sci-Fi, Thriller"
1,Bright,"Action, Fantasy, Thriller"
2,Avengers: Age of Ultron,"Action, Adventure, Sci-Fi"
3,Annihilation,"Adventure, Drama, Horror, Mystery, Sci-Fi, Thr..."
4,Hot Fuzz,"Action, Comedy, Mystery, Thriller"
5,Deadpool,"Action, Adventure, Comedy, Sci-Fi"
6,Bird Box,"Drama, Horror, Sci-Fi"
7,FYRE: The Greatest Party That Never Happened,"Documentary, Music"
8,The Big Short,"Biography, Comedy, Drama, History"
9,The Hitman's Bodyguard,"Action, Comedy, Crime, Thriller"


### 1.2 Minhash Signatures 
Using the movie genre and user_ids, try to implement your min-hash signatures so that users with similar interests in a genre appear in the same bucket. 

__Important note:__ You must write your minhash function from scratch.  You are not permitted to use any already implemented hash functions.  Read the class materials and, if necessary, conduct an internet search.  The description of hash functions in the [book](http://infolab.stanford.edu/~ullman/mmds/ch3n.pdf) may be helpful as a reference.


In [5]:
df.head()

,Unnamed: 0,datetime,duration,title,genres,release_date,movie_id,user_id
0,58773,2017-01-01 01:15:09,0.0,"Angus, Thongs and Perfect Snogging","Comedy, Drama, Romance",2008-07-25,26bd5987e8,1dea19f6fe
1,58774,2017-01-01 13:56:02,0.0,The Curse of Sleeping Beauty,"Fantasy, Horror, Mystery, Thriller",2016-06-02,f26ed2675e,544dcbc510
2,58775,2017-01-01 15:17:47,10530.0,London Has Fallen,"Action, Thriller",2016-03-04,f77e500e7a,7cbcc791bf
3,58776,2017-01-01 16:04:13,49.0,Vendetta,"Action, Drama",2015-06-12,c74aec7673,ebf43c36b6
4,58777,2017-01-01 19:16:37,0.0,The SpongeBob SquarePants Movie,"Animation, Action, Adventure, Comedy, Family, ...",2004-11-19,a80d6fc2aa,a57c992287


### Idea for solving 

First of all, let's create a dataframe with all genres that have been watched by unique users. So, we can use it for further calculations and for creation of a signature matrix.

In [6]:
# df_lsh will contain only user_id and genres
df_lsh = df.loc[:,['user_id', 'genres']] 

In [7]:
stemmer = PorterStemmer()
#drop potential null values from the description column
df_lsh = df_lsh.dropna(subset=['genres'])
#uses apply method with list comprehension to tokenize each row and stem each genre
df_lsh['genres_clean'] = df_lsh.genres.apply(lambda row: [stemmer.stem(word) for word in nltk.word_tokenize(row)])

In [8]:
# I noticed that there are some redundant words and symbols, which I will take away
remove_words = [',', 'avail', 'not']
df_lsh.genres_clean = df_lsh.genres_clean.apply(lambda row: [word for word in row if word not in remove_words]) 

In [9]:
# Let's add unique genres to out vocabulary set
vocabulary = set()
df_lsh.genres_clean.apply(lambda row: [vocabulary.add(word) for word in row]) 

0                           [None, None, None]
1                     [None, None, None, None]
2                                 [None, None]
3                                 [None, None]
4         [None, None, None, None, None, None]
                          ...                 
671731                                  [None]
671732          [None, None, None, None, None]
671733                      [None, None, None]
671734                            [None, None]
671735                            [None, None]
Name: genres_clean, Length: 671736, dtype: object

In [10]:
df_lsh.head()

,user_id,genres,genres_clean
0,1dea19f6fe,"Comedy, Drama, Romance","[comedi, drama, romanc]"
1,544dcbc510,"Fantasy, Horror, Mystery, Thriller","[fantasi, horror, mysteri, thriller]"
2,7cbcc791bf,"Action, Thriller","[action, thriller]"
3,ebf43c36b6,"Action, Drama","[action, drama]"
4,a57c992287,"Animation, Action, Adventure, Comedy, Family, ...","[anim, action, adventur, comedi, famili, fantasi]"


### Notice

In the above dataframe we already have a list of genres that user watched, but they are not unique, thus, let us use grouby with sum, so the lists of the same users will be added together.

In [11]:
df_lsh = df_lsh.groupby(by='user_id').agg({'genres_clean': 'sum'})

In [12]:
# Let's remove repeated values from the lists just by converting them to set
df_lsh.genres_clean = df_lsh.genres_clean.apply(lambda row: set(row))

In [13]:
df_lsh = df_lsh.reset_index()

In [14]:
# We have to convert vocabulary and the watched genres by the users again to the list, so we can easily work further
vocabulary = list(vocabulary)
df_lsh.genres_clean = df_lsh.genres_clean.apply(lambda row: list(row))

In [15]:
# Here is the final df_lsh, that has unique user ids with all the genres that was watched by the particular user.
df_lsh.head()

,user_id,genres_clean
0,00004e2862,"[crime, drama, thriller]"
1,000052a0a0,"[comedi, horror, drama, mysteri, fantasi, adve..."
2,000090e7c8,"[mysteri, thriller, sci-fi]"
3,000118a755,[horror]
4,000296842d,"[mysteri, thriller, sci-fi, drama]"


### Creation of the Signature Matrix

In order to create signature matrix, first we have to create shingles, the index of which will be hashed for creation of the signature matrix. So, in case of some sentences we could have used shingles of two characters, however, here we have specific information, which are genres. Thus, I have decided to use genres as my shingles. 

Let's use dictionary comprehension for creation of shingles with unique identifiers, they will serve as the row ids that will be hashed in the future.

In [16]:
shingle_dict = {genre: i for i, genre in enumerate(vocabulary)}

In [17]:
shingle_dict

{'comedi': 0,
 'drama': 1,
 'fantasi': 2,
 'adventur': 3,
 'film-noir': 4,
 'sci-fi': 5,
 'reality-tv': 6,
 'anim': 7,
 'mysteri': 8,
 'sport': 9,
 'thriller': 10,
 'crime': 11,
 'music': 12,
 'action': 13,
 'short': 14,
 'documentari': 15,
 'horror': 16,
 'war': 17,
 'western': 18,
 'famili': 19,
 'talk-show': 20,
 'new': 21,
 'romanc': 22,
 'biographi': 23,
 'histori': 24}

### Approach

In order to create a signature matrix, we do not need to use the one hot encoding vectores with the genres that every user has watched and permute them further. This is too time consuming. So, the idea is that we really should avoid matrix with most of the values that are zeros. We only care about the values that are ones. 

Therefore, we use shingle dict that was created above as the true row ids, then let's say that some particular user is watching thriller, sport and crime. We know their true row ids are 24, 23 and 20 respectively. We will use one has function for all users for creating the one row of the signature matrix, hence, we will hash those values of true row ids and take the minimum output from three values and put it to the signature matrix. 

Now, as the idea of using hash functions and shingle dict is clear, let's explain my approach. I will use 10 different hash functions to create 10 outputs for some particular user's genre, update the signature matrix and go to the next genre of that user and so on untill I finish with one user. This, then iteratively performed on all other users.

In [18]:
N = len(shingle_dict)
n_sig = 10
# Our hash will be (a*x + b) % N, where a and b are random numbers in range of N
# Here params will be a and b for all 10 hash functions
params = np.random.randint(N, size=[n_sig,2])

In [19]:
# We will use this hash function, to hash the row ids
# The hash function is basically (a*x + b) % N
def _permuteRow(row):
    return (params@np.array([1, row]))%N

In [20]:
# Let's create a signature matrix with all values as infinity, we make them infinity, 
# so they will be changed with the outputs from the hash functions 
# notice that in our signature matrix, columns are users and the rows are the outputs from the unique
# hash functions
sig = np.full((n_sig, df_lsh.shape[0]), np.inf)

In [21]:
# This is the core in creation of the signature matrix, it will go through our previously created 
# df_lsh, for each user it will update the values in signature matrix, leaving the minimum one. 
for j, row in df_lsh.iterrows():
    for shingle in row['genres_clean']:
        orig_row = shingle_dict[shingle]
        curr_col = _permuteRow(orig_row)
        sig[:,j] = np.minimum(sig[:,j],curr_col)

In [22]:
# Convert all the values in signature matrix to integer type
sig = sig.astype(int)
# Let's visualize the signature matrix
pd.DataFrame(sig)

/tmp/ipykernel_42936/1918318916.py:2: RuntimeWarning: invalid value encountered in cast
  sig = sig.astype(int)


,0,1,2,3,4,5,6,7,8,9,...,161908,161909,161910,161911,161912,161913,161914,161915,161916,161917
0,8,1,4,3,4,8,7,4,2,1,...,6,1,5,1,13,6,4,0,1,8
1,5,0,10,5,5,5,0,10,0,10,...,0,0,10,0,5,5,5,0,10,5
2,4,4,5,19,5,0,17,6,7,6,...,0,4,15,7,14,4,6,0,5,4
3,14,0,0,9,0,5,23,0,5,0,...,1,0,5,2,19,7,0,1,2,14
4,10,2,9,13,9,4,16,5,2,5,...,0,5,2,5,8,0,10,0,0,18
5,10,1,3,5,3,1,19,1,1,1,...,1,1,1,1,15,3,16,1,1,10
6,14,0,18,4,14,11,10,13,7,1,...,3,1,7,1,19,3,14,3,1,14
7,12,2,2,12,2,2,7,17,2,2,...,2,2,2,2,12,2,12,2,2,12
8,16,0,2,11,2,16,16,8,4,8,...,0,7,5,14,6,2,8,0,2,16
9,15,0,13,0,13,3,2,3,1,3,...,2,3,1,2,5,8,13,3,3,15


### 1.3 Locality-Sensitive Hashing (LSH)

Now that your buckets are ready, it's time to ask a few queries. We will provide you with some user_ids and ask you to recommend at __most five movies__ to the user to watch based on the movies clicked by similar users. 

To recommend at most five movies given a user_id, use the following procedure: 

1. Identify the <ins>two most similar</ins> users to this user.
2. If these two users have any movies __in common__, recommend those movies based on the total number of clicks by these users.
3. If there are __no more common__ movies, try to propose the most clicked movies by the __most similar user first__, followed by the other user. 

__Note:__ At the end of the process, we expect to see at most five movies recommended to the user.

I have used two sources from the internet, which were really useful. They can be found [here](https://towardsdatascience.com/locality-sensitive-hashing-how-to-find-similar-items-in-a-large-set-with-precision-d907c52b05fc) and [here](https://www.pinecone.io/learn/series/faiss/locality-sensitive-hashing/).

### Approach for creating buckets

Basically, the following lines of code below, put similar candidates to the same bucket. This is achieved using defaultdict with set values. We start going through columns which are unique users in some particular band, and our keys in defaultdict are column numbers from the signature matrix. Thus, we try to maximize the number of colissions, is two exactly similar columns are encountered they are certainly put to the same bucket, in out case, buckets are simply sets. Defaultdict is used for the case if there is no any key with that value, it will create one without throwing an exception.

In [23]:
def fastCandidatePairs(sig_mat, b, r):
    n, d = sig_mat.shape
    
    hashbuckets = collections.defaultdict(set)
    bands = np.array_split(sig_mat, b, axis=0)
    for i, band in enumerate(bands):
        for j in range(d):
            # The last value made a string to prevent colissions between bands
            band_id = tuple(list(band[:,j])+[str(i)])
            hashbuckets[band_id].add(j)
    bucket_candidates = list()
    for bucket in hashbuckets.values():
        if len(bucket) > 1:
            bucket_candidates.append(bucket)
    return bucket_candidates

In [24]:
sig.shape

(10, 161918)

In [25]:
candidate_pairs = fastCandidatePairs(sig, b=2, r=5)

### Choosing band `b` and row `r`

To choose `b` and `r`, we have to lean to the formula from the theory:

The threshold \( t \) for the Locality-Sensitive Hashing (LSH) theorem, with respect to bands \( b \) and rows \( r \), is given by:

$$
t = (1/b)^{1/r}
$$

Where b times r  is equal to the total number of hash functions used.
In our case, when \( b = 2 \) bands and \( r = 5 \) rows, the threshold \( t \) would be:

$$
t = (1/2)^{1/5} \approx 0.8706 
$$

This value of \( t \) determines the similarity threshold for hashing to the same bucket in at least one of the bands.



In [26]:
# After getting our buckets, we need to some score to evaluate which user is more similar than other one
# Here, I am using simple Jaccard Similarity, which is basically intersection over union.
def score(target_user_id, current_user_id, df):
    current_user = df.genres_clean.iloc[current_user_id]
    target_user = df.genres_clean.iloc[target_user_id]
    return len(set(target_user).intersection(set(current_user)))/len(set(target_user).union(set(current_user)))


In [27]:
def query(candidate_pairs, query_results, user_ids, df):
    # iterate over given user_ids
    for user_id in user_ids:
        # I need to match user_ids to indecies of my df_lsh, which I used to hash and 
        # perfrom other operations
        target_user_id = df.loc[df.user_id == user_id].index[0]
        # declare max_heap for keepint top-2 scores 
        max_heap = []
        # Iterate through buckets 
        for bucket in candidate_pairs:
            # Go further if there is out target user for whom we are looking two most similar candidates
            if target_user_id in bucket:

                # Iterate through users
                for current_user_id in bucket:
                    # Check if our user is not a target user, if not go further
                    if current_user_id != target_user_id:
                        # get a score
                        current_score = score(target_user_id, current_user_id, df)
                        # push the tuple consisting of current score and user_id
                        # check if the (current_score, current_user_id) is not already in the max_heap
                        # which could be the case, because we have not only one band
                        if (current_score, current_user_id) not in max_heap:
                            heapq.heappush(max_heap, (current_score, current_user_id))
                        if len(max_heap) > 2:
                            # remove the minimum if the length of 
                            # the heap larger than 2
                            heapq.heappop(max_heap)
        # put the two most simlar candidates after going through all buckets
        # it is important because target users could be in other buckets too!
        # and our job is to find two most similar candidates across all buckets
        query_results[target_user_id] = max_heap
    return query_results

### Query for finding two most similar users

Below, few lines of the code will go through some dummy user_ids and will recommend movies for them.

In [28]:
# let's take some user_ids
df.user_id[100]

'ad08fad2ec'

In [29]:
user_ids = [df.user_id[100], df.user_id[140000], df.user_id[51478]]

In [30]:
lsh_results = query(candidate_pairs, {}, user_ids, df_lsh)

In [31]:
lsh_results

{109317: [(1.0, 160034), (1.0, 160180)],
 89168: [(0.8, 17496), (0.8, 155288)],
 121616: [(1.0, 155637), (1.0, 157204)]}

### Creating Recommendations

After acquiring our results as a dictionary with keys as a target users for whom we are finding similar users and values as two tuples with the score and row id of the corresponding user we have to make recommendations for each user.



In [32]:
# Let us create an empy dataframe where we will put our recommendations for each user
recommendations_df = pd.DataFrame(columns=['user_id', 'recommended_movies'])

# Go through each user
for user_id in user_ids:
    recommended_movies = []

    # Identify two most similar users
    (_, second), (_, first) = lsh_results[df_lsh.loc[df_lsh.user_id == user_id].index[0]]
    
    # Let's find the movies that were already watched by the target user,
    # to not accidentally reccommend those movies
    df_target_movies = set(df.loc[df.user_id == user_id].title)
    # DataFrames for the first and second users
    df_first = df.loc[df.user_id == df_lsh.user_id.iloc[first]]
    df_second = df.loc[df.user_id == df_lsh.user_id.iloc[second]]

    # Sets of movies for each user
    movies_first = set(df_first.title)
    movies_second = set(df_second.title)

    # Find common movies
    common_movies = movies_first.intersection(movies_second)
    common_movies = [movie for movie in common_movies if movie not in df_target_movies]

    # Go further if there are some common movies
    if common_movies:
        # concatenate df_second under df_first
        combined_df = pd.concat([df_first, df_second])
        # filter by common movies
        combined_df = combined_df[combined_df.title.isin(common_movies)]
        # count the number of times each title was watched and sort them in descending order
        combined_df = combined_df.groupby('title').agg({'user_id': 'count'}).reset_index().sort_values(by='user_id', ascending=False)
        # add those movies 
        recommended_movies.extend(combined_df.title.tolist())
    if len(recommended_movies) >= 5:
        recommended_movies = recommended_movies[:6]
    else:
        # Add movies from each user if needed
        for df_user in [df_first, df_second]:
            if len(recommended_movies) < 5:
                top_movies = df_user.groupby('title').agg({'user_id':'count'}).reset_index().sort_values(by='user_id', ascending=False)
                for movie in top_movies.title:
                    # check some edge cases
                    if len(recommended_movies) < 5 and movie not in recommended_movies and movie not in df_target_movies:
                        recommended_movies.append(movie)
    
    new_row = pd.DataFrame({'user_id': [user_id], 'recommended_movies': [recommended_movies]})
    recommendations_df = pd.concat([recommendations_df, new_row], ignore_index=True)

### Analyzing results

Below, we have a dataframe with the recommendation movies for each of the target user. 
Let's try to analyze our results, and see if the recommendations are legitimate.

In [33]:
recommendations_df

,user_id,recommended_movies
0,ad08fad2ec,"[The Spy Who Dumped Me, Role Models]"
1,8d438090c0,"[The Longest Week, Mudbound, Pride, Raising th..."
2,c078c3bb09,"[Before I Fall, The Overnight, Here Alone, Lov..."


In [34]:
df.loc[df.user_id == "8d438090c0"]

,Unnamed: 0,datetime,duration,title,genres,release_date,movie_id,user_id
134067,192840,2017-07-29 15:42:36,103533.0,The Incredible Jessica James,Comedy,2017-07-28,f4f5186800,8d438090c0
134842,193615,2017-07-30 20:28:09,0.0,Handsome Devil,"Comedy, Drama, Sport",2017-06-02,cf85ceead2,8d438090c0
134963,193736,2017-07-30 22:56:56,9784.0,Le Week-End,"Comedy, Drama, Romance",2013-10-11,50605a2238,8d438090c0
135488,194261,2017-07-31 01:40:00,4750.0,The Accidental Husband,"Comedy, Romance",2008-02-29,12d726e81f,8d438090c0
135639,194412,2017-07-31 18:54:51,0.0,An Unfinished Life,"Drama, Family, Romance",2005-09-16,fe0a7072f2,8d438090c0
135652,194425,2017-07-31 02:59:10,0.0,Les Misérables,"Drama, History, Musical, Romance, War",2012-12-25,99e99b992b,8d438090c0
135729,194502,2017-07-31 03:09:10,0.0,Chef,"Adventure, Comedy, Drama",2014-05-30,3ba55497d3,8d438090c0
136047,194820,2017-08-01 15:00:27,11115.0,The Virgin Suicides,"Drama, Romance",2000-05-19,ef19c6dbea,8d438090c0
138338,197111,2017-08-04 12:01:16,0.0,Dallas Buyers Club,"Biography, Drama",2013-11-22,f28dd5526d,8d438090c0
140000,198773,2017-08-06 19:02:20,5258.0,Be Somebody,"Comedy, Drama, Romance",2016-06-10,6550a34aec,8d438090c0


In [35]:
# create a list of movies recommended for the user 8d438090c0
movies = recommendations_df[recommendations_df.user_id == "8d438090c0"].recommended_movies.tolist()[0]

In [36]:
df.loc[df.title.isin(movies)].groupby(by=['title', 'genres']).size().reset_index(name='count')

,title,genres,count
0,Bring It On: All or Nothing,"Comedy, Sport",186
1,Mudbound,"Drama, War",459
2,Mudbound,NOT AVAILABLE,31
3,Pride,"Biography, Comedy, Drama, History, Romance",187
4,Raising the Bar,"Drama, Family, Sport",183
5,The Longest Week,"Comedy, Drama, Romance",73


#### Conclusion

As it can be seen from the dataframes above, the recommended movies make sense, they match on genres and also none of the recommended movies was watched by the target user, which is good. Overall, LSH algorithm was successfully implemented, even though the number of signatures were not so big, because of the limited number of genres. 
